<a href="https://colab.research.google.com/github/gracewindheim/gmwDataSci/blob/master/q4hw8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question 4

Fit a multilayer neural network model on the SHHS data to predict whether rdi4p > 7 using the variables from the previous HW assigment. Compare the test set error with the penalized model from your previous HW.

In [0]:
# X variables = waist, gender, sex, bmi (gender, age_s1, bmi_s1, rdi4p)
# y outcome = if rdi4p > 7

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow import keras


In [0]:
! if [ ! -e oasis.csv ]; \
then wget https://raw.githubusercontent.com/gracewindheim/gmwDataSci/master/shhs1.txt; \
fi;

## Read in the data and display a few rows
dat = pd.read_csv("shhs1.txt", sep = "\t")
dat = dat.dropna()

--2020-04-28 22:13:48--  https://raw.githubusercontent.com/gracewindheim/gmwDataSci/master/shhs1.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 740206 (723K) [text/plain]
Saving to: ‘shhs1.txt.1’

shhs1.txt.1         100%[===================>] 722.86K  --.-KB/s    in 0.07s   

2020-04-28 22:13:48 (10.8 MB/s) - ‘shhs1.txt.1’ saved [740206/740206]



,pptid,waist,COPD15,ASTHMA15,slp_lat,time_bed,timest1p,timest2p,times34p,timeremp,rdi4p,StLOutP,StOnsetP,SlpPrdP,Staging1,Staging2,Staging3,Staging4,Staging5,RestAn1,RestAn2,RestAn3,RestAn4,HTNDerv_s1,shhs1_tcvd,gender,age_s1,smokstat_s1,WASO,bmi_s1
0,1,86.0,0.0,0.0,NaN,440.5,6.258322,60.852196,19.307590,13.581891,1.438083,28,28,375.5,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1,0,1,55,2.0,65.0,21.777553
1,2,107.0,0.0,0.0,NaN,225.0,0.824176,65.659340,16.758242,16.758242,17.802198,0,0,182.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1,78,0.0,43.0,32.950680
2,3,82.0,0.0,0.0,NaN,431.5,4.881451,40.306835,42.817295,11.994421,4.853556,167,167,358.5,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,0,77,0.0,73.0,24.114150
3,4,85.0,0.0,0.0,14.0,358.5,2.990033,29.401993,52.325581,15.282393,0.797342,54,82,301.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,0,1,48,0.0,43.5,20.185185


In [0]:
# Create binary variable that assigns if rdi4p > 7
dat['rdi4pBin'] = np.where(dat['rdi4p'] > 7, 1, 0)

# isolate x and y variables
dataset = dat[['bmi_s1'] + ['age_s1'] + ['gender'] + ['waist'] + ['rdi4pBin']]

# split data
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

## These commands operate on the training and testing
## data to remove the y outcome column and save it
train_outcome = train_dataset.pop('rdi4pBin')
test_outcome = test_dataset.pop('rdi4pBin')

train_stats = train_dataset.describe().transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
bmi_s1,2318.0,28.133346,5.063205,18.0,24.67128,27.44604,30.791183,50.0
age_s1,2318.0,62.973253,10.992687,39.0,55.00000,63.00000,71.000000,90.0
gender,2318.0,0.485764,0.499905,0.0,0.00000,0.00000,1.000000,1.0
waist,2318.0,96.705854,13.527326,67.0,88.00000,96.00000,105.000000,135.0


In [0]:
# normalize data
mn = train_stats['mean']
sd = train_stats['std']
normed_train_data = train_dataset.apply(lambda row: (row - mn) / sd, axis = 1)
normed_test_data = test_dataset.apply(lambda row: (row - mn) / sd, axis = 1)

In [0]:
# make data floats
x_train = normed_train_data.astype('float')
x_test = normed_test_data.astype('float')

y_train = train_outcome
y_test = test_outcome

(2318, 4)

In [0]:
# build model
model = keras.Sequential([
  keras.layers.Dense(64, activation='relu', input_dim=4),
  keras.layers.Dense(64, activation='relu'),
  keras.layers.Dropout(0.2),
  keras.layers.Dense(64, activation='relu'),
  keras.layers.Dropout(0.2),
  keras.layers.Dense(1)
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
training = model.fit(x_train, y_train, epochs = 100)

Epoch 1/100
73/73 [==============================] - 0s 1ms/step - loss: 0.7661 - accuracy: 0.6462
Epoch 2/100
73/73 [==============================] - 0s 2ms/step - loss: 0.6857 - accuracy: 0.6812
Epoch 3/100
73/73 [==============================] - 0s 2ms/step - loss: 0.6501 - accuracy: 0.6795
Epoch 4/100
73/73 [==============================] - 0s 2ms/step - loss: 0.6498 - accuracy: 0.6816
Epoch 5/100
73/73 [==============================] - 0s 2ms/step - loss: 0.6246 - accuracy: 0.6756
Epoch 6/100
73/73 [==============================] - 0s 1ms/step - loss: 0.6193 - accuracy: 0.6700
Epoch 7/100
73/73 [==============================] - 0s 2ms/step - loss: 0.6210 - accuracy: 0.6851
Epoch 8/100
73/73 [==============================] - 0s 1ms/step - loss: 0.5993 - accuracy: 0.6678
Epoch 9/100
73/73 [==============================] - 0s 2ms/step - loss: 0.5812 - accuracy: 0.6894
Epoch 10/100
73/73 [==============================] - 0s 2ms/step - loss: 0.6072 - accuracy: 0.6976
Epoch 11/

In [0]:
model.evaluate(x_test,  y_test, verbose=2)

19/19 - 0s - loss: 0.5708 - accuracy: 0.7168


[0.5707707405090332, 0.7167530059814453]

## Comparison
Multilayer neural network yielded 71.7% accuracy. My penalized model from last HW yielded an accuracy of around 81.2%.